Drive Connection


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Libraries

In [ ]:

import numpy as np
import pandas as pd 
import os
!pip install librosa
import librosa
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
          
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

Dataset 

In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/My Drive/coswara.csv")
from sklearn.utils import shuffle
#suffle the dataset
data = shuffle(data)
data=data.transpose()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [ ]:
# Pick up the labels and data
X = data.drop(data[[67415]], axis=1)
y=data[[67415]]

In [ ]:
#filling missing value if any row has
X.fillna(0, inplace = True)

In [ ]:
#Preprocessing
scaler = StandardScaler()
import numpy as np
X = scaler.fit_transform(np.array(X, dtype=np.float32))
#X=np.array(X, dtype=np.float32)

In [1]:
#coverting label into numeirc
encoder1 = LabelEncoder()
Y = encoder1.fit_transform(y)
#Y=np.array(Y,dtype=float)

In [ ]:
#applied PCA having a varience of 90%
pca = PCA()
principal_components12 = pca.fit_transform(X)
principal_df12 = pd.DataFrame(data = principal_components12)
principal_df12.shape

In [ ]:
#feature selection
#from sklearn.feature_selection import SelectKBest, chi2
#X_new = SelectKBest(chi2, k=20).fit_transform(X, y)

Transfomer coding

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
    if mask is not None:
        logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(logits, axis=-1)

    return tf.matmul(attention_weights, value)

MultiHeadAttention


In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 1,2,3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 1,2,3])

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        outputs = self.dense(concat_attention)

        return outputs

Positional Encoding

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :], d_model=d_model)
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

Padding_mask

In [ ]:
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

Encoder


In [ ]:
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
    if projection=='linear':
        projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
        print('linear')
  
    else:
        projection=tf.identity(inputs)
        print('none')
   
    projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    projection = PositionalEncoding(time_steps, d_model)(projection)

    outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
            )([outputs, padding_mask])
 
 
  

 
    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)


transformer main code


In [ ]:
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.float32))
  

    enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder')(inputs=[inputs, enc_padding_mask])

    #We reshape for feeding our FC in the next step
    outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
    #We predict our class
    outputs = tf.keras.layers.Dense(units=output_size,use_bias=True,activation='sigmoid', name="outputs")(outputs)

    return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

Spliting the dataset

In [ ]:
#We get our train and test set
X_train,X_test, Y_train, Y_test =train_test_split(principal_df12,Y, test_size=0.2, random_state=27)

In [ ]:
#dimesnion expanding because it required 3D but we are deailting with 2D
X_test= np.expand_dims(X_test, 1)
X_train= np.expand_dims(X_train, 1)

In [ ]:
principal_df12= np.expand_dims(principal_df12, 1)
#X= np.expand_dims(X, 1)

In [ ]:
#just for number of head it required if 0< then we required

In [ ]:
#projection=['linear','none']
projection=['linear']
accuracy=[]
proj_implemented=[]

Model for COVID-19



In [ ]:

for i in projection:
    NUM_LAYERS = 2
    D_MODEL = principal_df12.shape[2]
    NUM_HEADS = 1
    UNITS = 1024
    DROPOUT = 0.1
    TIME_STEPS= principal_df12.shape[1]
    OUTPUT_SIZE=1
    EPOCHS = 100
    EXPERIMENTS=1
     #num_layers = 4
    #d_model = 128
    #dff = 512
    #num_heads = 8
    #division   dropout_rate = 0.1
    for j in range(EXPERIMENTS):
        model = transformer(time_steps=TIME_STEPS,
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        output_size=OUTPUT_SIZE,  
        projection=i)
    
        model.compile(optimizer=tf.keras.optimizers.Adam(0.000001), loss='binary_crossentropy', metrics=['accuracy'])
        history=model.fit(X_train,Y_train, epochs=EPOCHS, validation_split=0.15)
        accr = model.evaluate(X_test,Y_test)
        print(accr)
        accuracy.append(max(history.history['val_accuracy']))
        proj_implemented.append(i)
        import matplotlib.pyplot as plt
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']

        epochs = range(1, len(acc) + 1)

        # "bo" is for "blue dot"
        plt.plot(epochs, loss, 'bo', label='Training loss')


        # b is for "solid blue line"
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.show()
        plt.clf()   # clear figure
        history_dict = history.history

        print(history_dict.keys())
        acc_values = history_dict['accuracy']
        val_acc_values = history_dict['val_accuracy']
        plt.plot(epochs, acc, 'bo', label='Training acc')
        plt.plot(epochs, val_acc, 'b', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')  
        plt.ylabel('Loss')
        plt.legend()

        plt.show()
        import matplotlib.pyplot as plt
        yhat_classes = model.predict(X_test,verbose=0)
                   
# predict crisp classes for test set
        #yhat_classes = model.predict_classes(X_train,verbose=0)
        #print(yhat_classes)
        #yhat_classes=yhat_classes.astype('int32')
        #yhat_classes=yhat_classes.reshape(-1,1)
        #yhat_probs = model.predict(X_test)
        #print(yhat_probs.round())
        #yhat_probs=yhat_probs.round()
        #yhat_classes=yhat_classes.reshape(-1,1)
    

        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(Y_test, yhat_classes.round())
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(Y_test, yhat_classes.round())
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(Y_test, yhat_classes.round())
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(Y_test, yhat_classes.round())
        print('F1 score: %f' % f1)

        # confusion matrix
        matrix = confusion_matrix(Y_test, yhat_classes.round())
        print(matrix)
        import seaborn as sns



        del model
        del history

In [ ]:
accuracy=pd.DataFrame(accuracy, columns=['accuracy'])
proj_implemented=pd.DataFrame(proj_implemented, columns=['projection'])
results=pd.concat([accuracy,proj_implemented],axis=1)

In [ ]:
results.groupby('projection').mean()
